In [1]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
anime_data = pd.read_csv('anime.csv')
rating_data = pd.read_csv('rating.csv')

In [3]:
anime_data.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
anime_data.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [5]:
#delete the anime ratings that are below 5
anime_data = anime_data[anime_data.rating > 5]
anime_data.describe()

,anime_id,rating,members
count,11001.000000,11001.000000,1.100100e+04
mean,12705.071175,6.678809,1.999690e+04
std,10903.815886,0.806112,5.758749e+04
min,1.000000,5.010000,1.300000e+01
25%,3051.000000,6.100000,3.410000e+02
50%,9095.000000,6.670000,1.985000e+03
75%,22071.000000,7.250000,1.179400e+04
max,34519.000000,10.000000,1.013917e+06


In [6]:
rating_data.tail()

,user_id,anime_id,rating
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9
7813736,73516,8074,9


In [4]:
# Remove animes with missing ratings (-1)
rating_data = rating_data[rating_data['rating'] != -1]

# Merge anime and rating data based on anime ID
merged_data = pd.merge(anime_data, rating_data, on='anime_id')

In [8]:
anime_data = []
rating_data = []

In [9]:
merged_data.head()

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,322,10


In [10]:
merged_data.describe()

,anime_id,rating_x,members,user_id,rating_y
count,6.325661e+06,6.325661e+06,6.325661e+06,6.325661e+06,6.325661e+06
mean,8.896002e+03,7.681474e+00,1.848707e+05,3.674916e+04,7.814839e+00
std,8.876977e+03,6.521717e-01,1.909977e+05,2.101331e+04,1.563512e+00
min,1.000000e+00,5.010000e+00,3.700000e+01,1.000000e+00,1.000000e+00
25%,1.238000e+03,7.290000e+00,4.688700e+04,1.898500e+04,7.000000e+00
50%,6.211000e+03,7.700000e+00,1.175650e+05,3.681500e+04,8.000000e+00
75%,1.407500e+04,8.160000e+00,2.563250e+05,5.487700e+04,9.000000e+00
max,3.447500e+04,9.370000e+00,1.013917e+06,7.351600e+04,1.000000e+01


In [11]:
merged_data.shape

(6325661, 9)

In [5]:
# delete all the users that rated less that 10 animes
merged_data = merged_data.groupby('user_id').filter(lambda x: len(x) >= 30)
merged_data.shape

(6025702, 9)

In [6]:
#delete all the users that sum of ratings is less than 50
merged_data = merged_data.groupby('user_id').filter(lambda x: x['rating_y'].sum() >= 200)
merged_data.shape

(6024582, 9)

In [7]:
#delete all the animes that have less than 50 ratings
merged_data = merged_data.groupby('anime_id').filter(lambda x: len(x) >= 1000)
merged_data.shape

(4879263, 9)

In [15]:
# rating y => user rating rating x => anime rating
ratings_matrix = merged_data.pivot_table(index='user_id', columns='anime_id', values='rating_y')
ratings_matrix.shape

(41373, 1437)

In [16]:
ratings_matrix.head()

anime_id,1,5,6,7,15,16,18,19,20,22,...,32281,32282,32379,32438,32542,32729,32828,32935,32998,34240
user_id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,8.0,NaN,6.0,NaN,6.0,NaN,6.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
merged_data = []

In [ ]:
# not the way that we are using right now
user_similarity = cosine_similarity(ratings_matrix.fillna(0))

In [61]:
user_similarity = []

In [18]:
# replace user id with index
ratings_matrix_index = ratings_matrix.reset_index(drop=True)
ratings_matrix_index.head()

anime_id,1,5,6,7,15,16,18,19,20,22,...,32281,32282,32379,32438,32542,32729,32828,32935,32998,34240
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,8.0,NaN,6.0,NaN,6.0,NaN,6.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
#find user with id 3 from ratings_matrix
user_id = 5
user = ratings_matrix.loc[user_id, :].values.reshape(1, -1)
user

array([[nan, nan,  8., ..., nan, nan, nan]])

In [19]:
#save rating matrix to csv
ratings_matrix.to_csv('ratings_matrix.csv', index=False)
ratings_matrix = []

In [112]:
#find user with id 42946 from ratings_matrix
user_id = 1
user = ratings_matrix_index.loc[user_id, :].values.reshape(1, -1)
user

array([[nan, nan,  8., ..., nan, nan, nan]])

In [20]:
ratings_matrix_index.to_csv('ratings_matrix_index.csv', index=False)

In [22]:
# Convert ratings matrix to sparse format
sparse_ratings = ratings_matrix_index.fillna(0).values

# Create Nearest Neighbors model
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(sparse_ratings)

NearestNeighbors(algorithm='brute', metric='cosine')

In [59]:
def get_similar_users(user_id, n_neighbors=5):
    # Find the nearest neighbors for the given user
    distances, indices = nn_model.kneighbors([sparse_ratings[user_id]], n_neighbors+1)
    # Exclude the first nearest neighbor (itself)
    distances = distances.flatten()[1:]
    indices = indices.flatten()[1:]
    return indices, distances

def make_recommendations(user_id, ratings_matrix, n_recommendations=5):
    similar_users, _ = get_similar_users(user_id)
    # print("similar_users", similar_users)
    user_ratings = ratings_matrix.loc[user_id].dropna()
    # print("user_ratings", user_ratings.dropna())
    recommendations = []

    
    for similar_user in similar_users:
        similar_user_ratings = ratings_matrix.loc[similar_user]
        # print("similar_user_ratings", similar_user_ratings)
        for anime_id, rating in similar_user_ratings.iteritems():
            if anime_id not in user_ratings and rating > 5:
                # print("anime_id", anime_id)
                recommendations.append((anime_id, rating))
    
    return recommendations

In [43]:
ratings_matrix_index.loc[1].dropna()

anime_id
6        8.0
15       6.0
18       6.0
20       6.0
22       5.0
        ... 
29803    8.0
30831    8.0
31240    8.0
32182    9.0
32828    7.0
Name: 1, Length: 367, dtype: float64

In [42]:
ratings_matrix_index.loc[34092].dropna()

anime_id
1        8.0
6        7.0
15       8.0
16       6.0
18       8.0
        ... 
31764    7.0
31964    8.0
32093    8.0
32182    9.0
32282    9.0
Name: 34092, Length: 434, dtype: float64

In [62]:
# Example usage
user_id = 1 # Example user ID
recommendations = make_recommendations(user_id, ratings_matrix_index, 5)

print("recommendations", recommendations)



recommendations [(1, 8.0), (16, 6.0), (28, 7.0), (33, 8.0), (44, 10.0), (60, 6.0), (61, 6.0), (71, 8.0), (72, 8.0), (73, 9.0), (74, 6.0), (79, 7.0), (102, 7.0), (120, 6.0), (121, 8.0), (135, 8.0), (154, 8.0), (161, 7.0), (164, 10.0), (177, 7.0), (187, 8.0), (210, 7.0), (227, 6.0), (237, 8.0), (270, 6.0), (317, 6.0), (330, 8.0), (355, 7.0), (356, 7.0), (372, 7.0), (401, 8.0), (430, 8.0), (431, 9.0), (433, 7.0), (457, 7.0), (462, 8.0), (464, 9.0), (469, 6.0), (479, 8.0), (523, 8.0), (558, 9.0), (587, 7.0), (627, 8.0), (658, 8.0), (777, 9.0), (849, 8.0), (857, 7.0), (889, 8.0), (901, 7.0), (953, 6.0), (969, 7.0), (1142, 6.0), (1250, 6.0), (1482, 8.0), (1519, 8.0), (1530, 6.0), (1536, 7.0), (1575, 9.0), (1604, 8.0), (1606, 7.0), (1686, 7.0), (1691, 8.0), (1698, 7.0), (1699, 7.0), (1722, 7.0), (1813, 7.0), (1842, 9.0), (2025, 8.0), (2026, 8.0), (2129, 6.0), (2167, 9.0), (2251, 9.0), (2593, 8.0), (2685, 7.0), (2759, 8.0), (2787, 7.0), (2889, 7.0), (2904, 8.0), (2924, 7.0), (2926, 7.0), (2927

content based RS with KNN


In [2]:
anime_data = pd.read_csv('anime.csv')
rating_data = pd.read_csv('rating.csv')
# handle null data
anime_data['type'] = anime_data['type'].fillna('None')
anime_data['genre'] = anime_data['genre'].fillna('None')
anime_data['rating'] = anime_data['rating'].fillna(anime_data['rating'].median())
# null data for episodes is Unknown so we replace it with np.nan
episode_ = anime_data['episodes'].replace('Unknown', np.nan)
episode_ = episode_.fillna(episode_.median())

anime_data.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

feature processing for future clustring

In [4]:
type_ = pd.get_dummies(anime_data['type'])
genre_ = anime_data['genre'].str.get_dummies(sep=',')
X = pd.concat([genre_, type_, episode_, anime_data['rating'], anime_data['members']],axis=1)
X.head()

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Movie,Music,None,ONA,OVA,Special,TV,episodes,rating,members
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,9.37,200630
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,64,9.26,793665
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,51,9.25,114262
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,24,9.17,673572
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,51,9.16,151266


In [5]:
# scale value in table to match other vals in other columns

scaled = MaxAbsScaler()
X = scaled.fit_transform(X)

In [11]:
# n_neighbors specify total number of recommendation -> we set it to 11 to get 10 similar anime
recommendations = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
recommendations.kneighbors(X)

anime_indices = recommendations.kneighbors(X)[1]

In [13]:
def get_index(x):
    # gives index for the anime
    return anime_data[anime_data['name']==x].index.tolist()[0]


In [14]:
def recommend(name):
    print('Here are 10 anime similar to', name, ':' '\n')
    index = get_index(name)


    for i in anime_indices[index][1:]:
            print(anime_data.iloc[i]['name'],
                  '\n' 'Genre: ', anime_data.iloc[i]['genre'],
                  '\n' 'Episode count: ', anime_data.iloc[i]['episodes'],
                  '\n' 'Rating out of 10:', anime_data.iloc[i]['rating'], '\n')

In [15]:
recommend("Monster")

Here are 10 anime similar to Monster :

Mousou Dairinin 
Genre:  Drama, Mystery, Police, Psychological, Supernatural, Thriller 
Episode count:  13 
Rating out of 10: 7.74 

Mayoiga 
Genre:  Drama, Horror, Mystery, Psychological 
Episode count:  12 
Rating out of 10: 5.8 

Death Parade 
Genre:  Drama, Game, Mystery, Psychological, Thriller 
Episode count:  12 
Rating out of 10: 8.33 

Aoi Bungaku Series 
Genre:  Drama, Historical, Psychological, Seinen, Thriller 
Episode count:  12 
Rating out of 10: 7.9 

Night Head Genesis 
Genre:  Drama, Horror, Mystery, Psychological, Supernatural 
Episode count:  24 
Rating out of 10: 6.88 

Narutaru: Mukuro Naru Hoshi Tama Taru Ko 
Genre:  Drama, Seinen, Thriller 
Episode count:  13 
Rating out of 10: 6.28 

Rainbow: Nisha Rokubou no Shichinin 
Genre:  Drama, Historical, Seinen, Thriller 
Episode count:  26 
Rating out of 10: 8.64 

Re:Zero kara Hajimeru Isekai Seikatsu 
Genre:  Drama, Fantasy, Psychological, Thriller 
Episode count:  25 
Rating o